In [93]:
import numpy as np
import pandas as pd
import difflib
import ast

In [94]:
data = pd.read_csv("/Users/caizhenghua/Desktop/40597446.csv")

In [95]:
bug_event  = "AppCrashEvent"
bug_time = "2022-07-23T13:47:35.000Z"
System_update_event = "Update.Orchestrator.WuProvider"
System_restart_event = "KernelGeneral.SystemStart"
Event_ls = ["AppHangEvent","AppCrashEvent","SystemStart","Update","Launch","Reboot","Click","Start"]

In [96]:
def nparrayTodict(data,i):
    data = ast.literal_eval(list(data)[i])
#     print(data, type(data))
    return data
def fuzzy_matching(st,cst):
    l_st = st.lower()
    cl_st = cst.lower()
    score = difflib.SequenceMatcher(None, l_st, cl_st).quick_ratio()
    if(l_st in cl_st):
        return True
    elif(score>=0.98):
        return True
    return False
def find_event_app(data,et):
    full_event_ls = []
    for event in list(data.name):
        if fuzzy_matching(et,event):
            full_event_ls.append(event)
    df1 = data.loc[data["name"].isin(full_event_ls)]
    df1 = df1[["name","appid","time","data"]]
    return df1
def find_app_event(data,appid):
    return data.loc[data["appid"]==appid]

In [97]:
ls1 = list(data["name"])
system_update_event_ls  = []
system_restart_event_ls = []
error_event_ls = []
#查询bug发生前是否有异常事件
error_event = ["ActivationManager.AAMShim_ActivationStop","Microsoft.Windows.Fundamentals.UserInitiatedFeedback.FeedbackHubError","Microsoft.Windows.WindowsErrorReporting.WerEvent","Microsoft.Windows.FaultReporting.AppCrashEvent","Microsoft.Windows.WindowsErrorReporting.WerEvent","Watson Failure: application_hang_busyhang_cffffff_atioglxx.dll!canevict"]
for i in ls1:
    if(fuzzy_matching(System_update_event,i) and i not in system_update_event_ls):
        system_update_event_ls.append(i)
    if(fuzzy_matching(System_restart_event,i) and i not in system_restart_event_ls):
        system_restart_event_ls.append(i)
for i in error_event:
    for j in ls1:
        if(i==j):
            error_event_ls.append(j)
            break
        if(fuzzy_matching(i,j)):
            error_event_ls.append(j)

In [98]:
error_event_ls,system_restart_event_ls,system_update_event_ls

(['ActivationManager.AAMShim_ActivationStop',
  'Microsoft.Windows.WindowsErrorReporting.WerEvent',
  'Microsoft.Windows.FaultReporting.AppCrashEvent',
  'Microsoft.Windows.WindowsErrorReporting.WerEvent'],
 ['KernelGeneral.SystemStart'],
 ['Microsoft.Windows.Update.Orchestrator.WuProvider.ScanService',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.Scan',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.Download',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.Stick',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.Install',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.Unstick',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.Revert',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.ActivityError',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.StorageReservesUsed',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.Commit',
  'Microsoft.Windows.Update.Orchestrator.WuProvider.PostReboot'])

In [99]:
# 给定数据，返回错误数据的条数
def judgeErrorEvent(trob_data):
    judge_ls = ["ProblemCode","HR","hresult","WinError"]
    error_ls = []
    crash_ls = []
    index = trob_data.index
    for i in range(len(trob_data)):
        data01 = nparrayTodict(trob_data["data"],i)
        com_key = [key for key in judge_ls if key in data01.keys()]
        for key in com_key:
#             if("AppName" in data01.keys()):
#                 if(fuzzy_matching("Unity",nparrayTodict(trob_data["data"],i)["AppName"]) and data01[key]!="0"):
# #                     print(i)
#                     error_ls.append(index[i])
            if(data01[key]!="0"):
                print(index[i])
                error_ls.append(index[i])
    for i in range(len(trob_data["name"])):
        if(fuzzy_matching(bug_event,trob_data["name"].iloc[i])):
            AppName = nparrayTodict(trob_data.iloc[[i],:].data,0)["AppName"]
#             print(AppName)
            if(fuzzy_matching("Unity",AppName)):
                crash_ls.append(index[i])
                print(index[i])
    return len(error_ls),len(crash_ls)

In [100]:
updateIndex = list(data.loc[data["name"].isin(system_update_event_ls)].index)
updateIndex.insert(0,0)
updateIndex.insert(len(data),len(data))
updateIndex = np.array(updateIndex)
updateIndex

array([    0,  4553,  4555,  4591,  4592,  4596,  4616,  4617,  4625,
        4671,  4679, 10030, 10031, 10493, 10494, 14598, 14600, 14734,
       14758, 14759, 14775, 14782, 14783, 14792, 14796, 16863, 16864,
       16874, 16875, 22849, 22850, 22860, 22874, 22875, 22878, 22883,
       22884, 22915, 22924, 27012, 27013, 27061, 27063, 27072, 27074,
       27079, 27122, 27127, 27128, 27179, 27185, 27486, 27487, 27489,
       27492, 27554, 27564, 28682, 28695, 28696, 28700, 31750, 31943,
       33589, 34259, 39127, 39128, 39955, 39965, 39966, 39974, 39975,
       39981, 39982, 39999, 40187, 40197, 50187, 50189, 50220, 50221,
       51881, 51883, 51892, 51900, 54587, 54588, 54722, 54738, 54739,
       54880, 54888, 54889, 54891, 55096, 57285, 57304, 59230, 59232,
       59290, 59291, 64323, 64324, 64367, 64380, 64381, 64388, 64391,
       64392, 64422, 64442, 81851, 81852, 82121, 82134, 82135, 82161,
       82164, 82165, 82268, 82278, 98104])

In [101]:
def calErrorEventNum(data):
    countError = 0
    countCrash = 0
#     updateIndex = data.loc[data["name"].isin(system_update_event_ls)].index
    for i in range(len(updateIndex)):
        if(i+1<len(updateIndex)):
            c1 = updateIndex[i]
            c2 = updateIndex[i+1]
            errorNum,crashNum = judgeErrorEvent(data.iloc[c1:c2,:])
            countError+=errorNum
            countCrash+=crashNum
            if(errorNum>0 or crashNum>0):
                print(f"updateindex:{updateIndex[i]},errorNum:{errorNum},crashNum:{crashNum},countError:{countError},countCrash:{countCrash}")
            else:
                print(f"{updateIndex[i]}分块无对应app的异常事件和crash事件")
            

    print(f"totalError:{countError},totalCrash:{countCrash}")
    
# print(errorNum,crashNum)

In [102]:
calErrorEventNum(data)

4104
4133
updateindex:0,errorNum:2,crashNum:0,countError:2,countCrash:0
4553分块无对应app的异常事件和crash事件
4555分块无对应app的异常事件和crash事件
4591分块无对应app的异常事件和crash事件
4592分块无对应app的异常事件和crash事件
4596分块无对应app的异常事件和crash事件
4616分块无对应app的异常事件和crash事件
4617分块无对应app的异常事件和crash事件
4625分块无对应app的异常事件和crash事件
4671分块无对应app的异常事件和crash事件
4679分块无对应app的异常事件和crash事件
10030分块无对应app的异常事件和crash事件
10031分块无对应app的异常事件和crash事件
10493分块无对应app的异常事件和crash事件
10494分块无对应app的异常事件和crash事件
14598分块无对应app的异常事件和crash事件
14600分块无对应app的异常事件和crash事件
14734分块无对应app的异常事件和crash事件
14758分块无对应app的异常事件和crash事件
14759分块无对应app的异常事件和crash事件
14775分块无对应app的异常事件和crash事件
14782分块无对应app的异常事件和crash事件
14783分块无对应app的异常事件和crash事件
14792分块无对应app的异常事件和crash事件
14796分块无对应app的异常事件和crash事件
16863分块无对应app的异常事件和crash事件
16864分块无对应app的异常事件和crash事件
16874分块无对应app的异常事件和crash事件
17826
20528
20530
20532
21058
updateindex:16875,errorNum:5,crashNum:0,countError:7,countCrash:0
22849分块无对应app的异常事件和crash事件
22850分块无对应app的异常事件和crash事件
22860分块无对应app的异常事件和crash事件
22874分块无对应app的异常事件和crash事件
22875

In [103]:
updateIndex = list(data.loc[data["name"]==system_update_event_ls[-1]].index)
updateIndex.insert(0,0)
updateIndex.insert(len(data),len(data))
updateIndex = np.array(updateIndex)
updateIndex

array([    0, 33589, 34259, 57285, 57304, 98104])

In [104]:
calErrorEventNum(data)

4104
4133
17826
20528
20530
20532
21058
25763
25765
25767
33465
updateindex:0,errorNum:11,crashNum:0,countError:11,countCrash:0
33589分块无对应app的异常事件和crash事件
37783
37785
37787
45612
45667
updateindex:34259,errorNum:5,crashNum:0,countError:16,countCrash:0
57285分块无对应app的异常事件和crash事件
66488
66490
66492
57869
59732
59743
60689
65358
68174
84747
89093
90430
90432
90452
90453
91905
91907
updateindex:57304,errorNum:3,crashNum:14,countError:19,countCrash:14
totalError:19,totalCrash:14
